In [6]:
%%spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# song_schema = StructType(\
#                          StructField("a", IntegerType, True) \
#                          StructField("b", LongType, False) \
#                          StructField("c", BooleanType, False) \
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# log_data_s3_uri = "s3://udacity-dend/log_data/"
# song_data_s3_uri = "s3://udacity-dend/song_data/"

# log_data_s3_uri = "s3a://udacity-bkt-rc301-spark-project/data/log_data"
# song_data_s3_uri = "s3a://udacity-bkt-rc301-spark-project/data/song_data/*/*/*"

# df_log = spark.read.json(log_data_s3_uri)
# df_song = spark.read.json(song_data_s3_uri)

In [ ]:
# df_song.printSchema()

In [ ]:
# df_log.show(10)
# df_log.printSchema()

## Process Song Data

In [ ]:
# df.schema

In [10]:
# song_data_s3_uri = "s3://udacity-dend/song_data/"
input_data = "s3a://udacity-bkt-rc301-spark-project/data/"
song_data = f"{input_data}song_data/*/*/*"
song_schema = StructType([StructField('artist_id', StringType(), True), \
                            StructField('artist_latitude', DoubleType(), True), \
                            StructField('artist_location', StringType(), True), \
                            StructField('artist_longitude', DoubleType(), True), \
                            StructField('artist_name', StringType(), True), \
                            StructField('duration', DoubleType(), True), \
                            StructField('num_songs', LongType(), True), \
                            StructField('song_id', StringType(), True), \
                            StructField('title', StringType(), True), \
                            StructField('year', IntegerType(), True) \
                         ])

df = spark.read.schema(song_schema).json(song_data)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)

In [ ]:
# schema = StructType([ \
#     StructField("song_id",StringType(),True), \
#     StructField("title",StringType(),True), \
#     StructField("artist_id",StringType(),True), \
#     StructField("year", IntegerType(), True), \
#     StructField("duration", DoubleType(), True)
#   ])

In [4]:
# extract columns to create songs table
cols_songs_table = ['song_id', 'title', 'artist_id', 'year', 'duration']
songs_table = df.select(cols_songs_table).dropDuplicates(cols_songs_table)
songs_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: double (nullable = true)

In [5]:
# songs_table.printSchema()
# print(songs_table.count())
# print(songs_table.dropDuplicates(cols_songs_table).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# write songs table to parquet files partitioned by year and artist
output_path = "s3://udacity-bkt-rc301-spark-project/output_data/songs_table/"
songs_table.write \
        .option("header", True) \
        .partitionBy("year", "artist_id") \
        .mode("Overwrite") \
        .parquet(output_path)

# another way of writing
# user_log.write.save(out_path, format="csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# extract columns to create artists table
cols_artists_table = [
    'artist_id', 'artist_name', 'artist_location', 'artist_latitude',
    'artist_longitude'
]
artists_table = df.select(cols_artists_table).dropDuplicates(cols_artists_table)
artists_table = artists_table.withColumnRenamed('artist_name', 'name') \
                .withColumnRenamed('artist_location', 'location') \
                .withColumnRenamed('artist_latitude','latitude') \
                .withColumnRenamed('artist_longitude','longitude')
artists_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [ ]:
# write artists table to parquet files
output_path = "s3://udacity-bkt-rc301-spark-project/output_data/artists_table/"
artists_table.write \
        .option("header", True) \
        .mode("Overwrite") \
        .parquet(output_path)

## Process Log Data

In [11]:
# parameters
input_data = 's3a://udacity-bkt-rc301-spark-project/data/'
output_data = 's3://udacity-bkt-rc301-spark-project/output_data/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
# get filepath to log data file
log_data = f"{input_data}log_data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
# read log data file
log_schema = StructType( \
                    [StructField('artist', StringType(), True), \
                         StructField('auth', StringType(), True), \
                         StructField('firstName', StringType(), True), \
                         StructField('gender', StringType(), True), \
                         StructField('itemInSession', LongType(), True), \
                         StructField('lastName', StringType(), True), \
                         StructField('length', DoubleType(), True), \
                         StructField('level', StringType(), True), \
                         StructField('location', StringType(), True), \
                         StructField('method', StringType(), True), \
                         StructField('page', StringType(), True), \
                         StructField('registration', DoubleType(), True), \
                         StructField('sessionId', LongType(), True), \
                         StructField('song', StringType(), True), \
                         StructField('status', LongType(), True), \
                         StructField('ts', LongType(), True), \
                         StructField('userAgent', StringType(), True), \
                         StructField('userId', StringType(), True) \
                    ])
df = spark.read.schema(log_schema)\
                .json(log_data) \
                .withColumnRenamed('userId', 'user_id') \
                .withColumnRenamed('firstName', 'first_name') \
                .withColumnRenamed('lastName','last_name') \
                .withColumnRenamed('sessionId','session_id') \
                .withColumnRenamed('userAgent','user_agent')
# df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
# filter by actions for song plays
df = df.filter("page=='NextSong'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
# extract columns for users table    
cols_users_table = ['user_id', 'first_name', 'last_name', 'gender', 'level']
users_table = df.select(cols_users_table).dropDuplicates(cols_users_table)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
# write users table to parquet files
users_table.write \
        .option("header", True) \
        .mode("Overwrite") \
        .parquet(f"{output_data}users_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
a = df.select('ts')
# a.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
# get_timestamp = udf(lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour), TimestampType())
get_timestamp = udf(lambda x : datetime.fromtimestamp(x / 1000), TimestampType())
b = a.withColumn('timestamp', get_timestamp('ts'))
b.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------------------+
|ts           |timestamp              |
+-------------+-----------------------+
|1542241826796|2018-11-15 00:30:26.796|
|1542242481796|2018-11-15 00:41:21.796|
|1542242741796|2018-11-15 00:45:41.796|
|1542253449796|2018-11-15 03:44:09.796|
|1542260935796|2018-11-15 05:48:55.796|
+-------------+-----------------------+
only showing top 5 rows

In [94]:
# c = a.select(to_timestamp(a.ts/1000).alias('utc_time'))#.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
# a.select(from_unixtime(a.ts/1000).alias('utc_time')).show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x : datetime.fromtimestamp(x / 1000), TimestampType())
df = df.withColumn('start_time', get_timestamp('ts'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- last_name: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- session_id: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- dt: struct (nullable = true)
 |    |-- hour: integer (nullable = true)
 |    |-- day: integer (nullable = true)
 |    |-- week: integer (nullable = true)
 |    |-- month: integer (nullable = true)
 |    |-- year: integer (nullable = true)
 |    |-- w

In [98]:
# tst = udf(lambda {x, x+1} : x)
# dt_type = StructType( \
#             [StructField('hour', IntegerType(), True), \
#                  StructField('day', IntegerType(), True) \
#             ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
# create datetime column from original timestamp column
dt_type = StructType( \
            [StructField('hour', IntegerType(), True), \
                 StructField('day', IntegerType(), True), \
                 StructField('week', IntegerType(), True), \
                 StructField('month', IntegerType(), True), \
                 StructField('year', IntegerType(), True), \
                 StructField('weekday', IntegerType(), True) \
            ])

def get_datetime_parts(ts):
    dt = datetime.fromtimestamp(ts / 1000)
    hour = dt.hour
    day = dt.day
    week = int(dt.strftime("%W"))
    month = dt.month
    year = dt.year
    weekday = dt.date().weekday()
    return hour, day, week, month, year, weekday

get_datetime = udf(get_datetime_parts, dt_type)
df = df.withColumn('dt', get_datetime('ts'))
# df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- last_name: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- session_id: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- dt: struct (nullable = true)
 |    |-- hour: integer (nullable = true)
 |    |-- day: integer (nullable = true)
 |    |-- week: integer (nullable = true)
 |    |-- month: integer (nullable = true)
 |    |-- year: integer (nullable = true)
 |    |-- 

In [101]:
# df.select('dt.week').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
# extract columns to create time table
time_table_cols = ['start_time', 'dt.hour', 'dt.day', 'dt.week', 'dt.month', 'dt.year', 'dt.weekday']
time_table = df.select(time_table_cols).distinct()
time_table.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----+---+----+-----+----+-------+
|start_time             |hour|day|week|month|year|weekday|
+-----------------------+----+---+----+-----+----+-------+
|2018-11-30 10:18:07.796|10  |30 |48  |11   |2018|4      |
|2018-11-30 10:35:18.796|10  |30 |48  |11   |2018|4      |
|2018-11-30 16:07:52.796|16  |30 |48  |11   |2018|4      |
|2018-11-16 14:09:01.796|14  |16 |46  |11   |2018|4      |
|2018-11-16 16:49:10.796|16  |16 |46  |11   |2018|4      |
+-----------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [103]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [104]:
# write time table to parquet files partitioned by year and month
time_table.write \
        .option("header", True) \
        .partitionBy("year", "month") \
        .mode("Overwrite") \
        .parquet(f"{output_data}time_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
# read in song data to use for songplays table
song_data = f"{input_data}song_data/*/*/*"
song_schema = StructType([StructField('artist_id', StringType(), True), \
                            StructField('artist_latitude', DoubleType(), True), \
                            StructField('artist_location', StringType(), True), \
                            StructField('artist_longitude', DoubleType(), True), \
                            StructField('artist_name', StringType(), True), \
                            StructField('duration', DoubleType(), True), \
                            StructField('num_songs', LongType(), True), \
                            StructField('song_id', StringType(), True), \
                            StructField('title', StringType(), True), \
                            StructField('year', IntegerType(), True) \
                         ])
song_cols = ['artist_id', 'artist_name', 'song_id', 'title']
song_df = spark.read.schema(song_schema).json(song_data).select(song_cols).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
# song_df.printSchema()
song_df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------------+------------------+-----------------------------+
|artist_id         |artist_name          |song_id           |title                        |
+------------------+---------------------+------------------+-----------------------------+
|AR10USD1187B99F3F1|Tweeterfriendly Music|SOHKNRJ12A6701D1F8|Drop of Rain                 |
|AR0IAWL1187B9A96D0|Danilo Perez         |SONSKXP12A8C13A2C9|Native Soul                  |
|AREDBBQ1187B98AFF5|Eddie Calvert        |SOBBXLX12A58A79DDA|Erica (2005 Digital Remaster)|
|AR62SOJ1187FB47BB5|Chase & Status       |SOGVQGJ12AB017F169|Ten Tonne                    |
|ARULZCI1241B9C8611|Luna Orbit Project   |SOSWKAV12AB018FC91|Midnight Star                |
+------------------+---------------------+------------------+-----------------------------+
only showing top 5 rows

In [147]:
# df.summary().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------+----------+------+------------------+---------+------------------+-----+--------------------+------+--------+--------------------+-----------------+--------------------+------+--------------------+--------------------+-----------------+
|summary|            artist|     auth|first_name|gender|     itemInSession|last_name|            length|level|            location|method|    page|        registration|       session_id|                song|status|                  ts|          user_agent|          user_id|
+-------+------------------+---------+----------+------+------------------+---------+------------------+-----+--------------------+------+--------+--------------------+-----------------+--------------------+------+--------------------+--------------------+-----------------+
|  count|              6820|     6820|      6820|  6820|              6820|     6820|              6820| 6820|                6820|  6820|    6820|                6820|       

In [173]:
# extract columns from joined song and log datasets to create songplays table 
log_cols = ['start_time', 'user_id', 'artist', 'song', 'level', 'session_id', 'location', 'user_agent', 'dt.year', 'dt.month']
# songplays_cols = ['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']

join_condition = ((df.artist==song_df.artist_name) & (df.song==song_df.title))
songplays_table = df.select(log_cols) \
    .join(song_df, on=join_condition, how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
# songplays_cols = ['start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent', 'year', 'month']
# songplays_table = songplays_table.select(songplays_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- song: string (nullable = true)
 |-- level: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)

In [176]:
songplays_table.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-------+-----------+----------------------------------------------+-----+----------+------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+----+-----+---------+-----------+-------+-----+
|start_time             |user_id|artist     |song                                          |level|session_id|location                            |user_agent                                                                                                                               |year|month|artist_id|artist_name|song_id|title|
+-----------------------+-------+-----------+----------------------------------------------+-----+----------+------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+----+-----+---------+-----------+-------+-----+
|201

In [177]:
window = Window.orderBy('start_time')
songplays_table = songplays_table.withColumn('songplay_id', row_number().over(window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
# songplays_table.dropDuplicates(['songplay_id']).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
# write songplays table to parquet files partitioned by year and month
songplays_cols_final = ['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent', 'year', 'month']
songplays_table.select(songplays_cols_final).write \
        .option("header", True) \
        .partitionBy("year", "month") \
        .mode("Overwrite") \
        .parquet(f"{output_data}songplays_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…